# *Öneri Sistemleri*

# 1. Basit Tavsiyeler

* #### Bu tavsiye sistemi filmlere verilen puanlara ve oy veren sayısına göre en iyi filmleri listeleyecektir. 
* #### Bunu yaparken ağırlıklı derecelendirme kullanılacaktır.
* #### Ağırlıklı derecelendirme formülü: **WeightedRating(WR) = ((v/(v+m)) * R) + ((m/(v+m)) * C)**
* #### **v** = filmin oy sayısı
* #### **m** = çizelgede listelenmesi gereken oylar
* #### **R** = filmin ortama puanı
* #### **C** = verinin tamamındaki ortalama oy

In [1]:
import pandas as pd 
import numpy as np

In [2]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv').drop("timestamp", axis = 1) # timestamp değişkenini kullanmayacağız

In [3]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [5]:
movie.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [6]:
rating.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [7]:
movie["movieId"] = movie["movieId"].astype(object)
rating["movieId"] = rating["movieId"].astype(object)
rating["userId"] = rating["userId"].astype(object)

In [8]:
movie.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [9]:
rating.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

* #### Veri üzerinde değişikliğe gidilmeden önce tüm oyların ortalamasını alıyoruz.

In [10]:
C = rating["rating"].mean()
C

3.5255285642993797

* #### Bir filmin ortalama derecesinin 5'lik bir ölçekte 3.52 civarında olduğunu görüyoruz.
* #### Öncelikle **rating** verisini film **Id**’lerine göre kaç tane kullanıcı tarafından kaç oy alındığına dair grupluyoruz. 
* #### Daha sonradan da index üzerinde düzenlemeler yapıyoruz.

In [11]:
rating_count = rating.groupby(["movieId"]).count()
rating_count["movieId"] = rating_count.index
rating_count.index.name = None
rating_count.reset_index(inplace = True)
del rating_count["index"]
rating_count

,userId,rating,movieId
0,49695,49695,1
1,22243,22243,2
2,12735,12735,3
3,2756,2756,4
4,12161,12161,5
...,...,...,...
26739,1,1,131254
26740,1,1,131256
26741,1,1,131258
26742,1,1,131260


* #### Şimdi de **rating** verisini film **Id**’lerine göre ortalamalarını veren yeni bir veri ortaya koyuyoruz. 
* #### Daha sonradan bu veri üzerinde de index düzenlemeleri yapıyoruz.

In [12]:
means = rating.groupby(["movieId"]).mean()
means["movieId"] = means.index
means.index.name = None
means.reset_index(inplace = True)
del means["index"]
means = means.rename(columns = {"rating":"mean"}) 
means

,mean,movieId
0,3.921240,1
1,3.211977,2
2,3.151040,3
3,2.861393,4
4,3.064592,5
...,...,...
26739,4.000000,131254
26740,4.000000,131256
26741,2.500000,131258
26742,3.000000,131260


* #### Şimdi ise elde ettiğimiz iki veriyi merge ile **movieId** değişkenine göre birleştiriyoruz.

In [13]:
df = pd.merge(movie, rating_count)
df = pd.merge(df, means)
df.head()

,movieId,title,genres,userId,rating,mean
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,49695,49695,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,22243,22243,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,12735,12735,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2756,2756,2.861393
4,5,Father of the Bride Part II (1995),Comedy,12161,12161,3.064592


* #### 90. persentilde bir filmin aldığı oy sayısını, **m** olarak hesaplıyoruz.

In [14]:
m = df["userId"].quantile(0.90)
m

1305.7000000000007

* #### Oylama sayısına göre uygun olan filmler;

In [15]:
q_movies = df.copy().loc[df["userId"] >= m]
q_movies.shape

(2675, 6)

* #### Bu listede 2675 tane film yer almaktadır. 
* #### Her nitelikli film için metrik hesaplanması gerekiyor. 
* #### Bunun için **weighted_rating()** tanımlayacak ve score bu işlemi nitelikli filmlere uygulayacak.
* #### Her filmin ağırlıklı derecelendirmesini hesaplayan işlev;

In [16]:
def weighted_rating(x, m=m, C=C):
    v = x["userId"]
    R = x["mean"]
    return (v/(v+m) * R) + (m/(m+v) * C)

* #### Yeni bir özellik **score** tanımlayıp değerini **weightted_rating()** ile hesaplıyoruz.
* #### Ardından veriyi bu özelliğe göre sıralıyoruz. 
* #### En iyi film sıralamasını ortaya çıkarıyoruz.

In [17]:
q_movies["score"] = q_movies.apply(weighted_rating, axis = 1)
q_movies = q_movies.sort_values("score", ascending = False)
q_movies.head(10)

,movieId,title,genres,userId,rating,mean,score
315,318,"Shawshank Redemption, The (1994)",Crime|Drama,63366,63366,4.446990,4.428386
843,858,"Godfather, The (1972)",Crime|Drama,41355,41355,4.364732,4.339047
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,47006,47006,4.334372,4.312512
523,527,Schindler's List (1993),Drama|War,50054,50054,4.310175,4.290227
1195,1221,"Godfather: Part II, The (1974)",Crime|Drama,27398,27398,4.275641,4.241519
895,912,Casablanca (1942),Drama|Romance,24349,24349,4.258327,4.221031
887,904,Rear Window (1954),Mystery|Thriller,17449,17449,4.271334,4.219411
1169,1193,One Flew Over the Cuckoo's Nest (1975),Drama,29932,29932,4.248079,4.217877
737,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,23220,23220,4.247287,4.208862
2873,2959,Fight Club (1999),Action|Crime|Drama|Thriller,40106,40106,4.227123,4.205002


# 2. İçeriğe Dayalı Tavsiyeler
* #### Bu kısımda da filmlerin türlerine göre benzer filmleri veren bir tavsiye sistemi uygulayacağız.

In [18]:
df[["genres"]].head()

,genres
0,Adventure|Animation|Children|Comedy|Fantasy
1,Adventure|Children|Fantasy
2,Comedy|Romance
3,Comedy|Drama|Romance
4,Comedy


* #### Terim Frekansı - Ters Belge Frekansı (TF-IDF) vektörlerini hesaplayacağız.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

* #### "**The**" ve "**a**" gibi verileri kaldıralım.
* #### Ardından **TF-IDF** matrisini oluşturalım.

In [20]:
tfidf = TfidfVectorizer(stop_words = "english")
df["genres"] = df["genres"].fillna("|")

tfidf_matrix = tfidf.fit_transform(df["genres"])
tfidf_matrix.shape

(26744, 23)

* #### 26744 film için 23 farklı tür olduğunu görüyoruz.
* #### Bu matris ile benzerlik skorunu **kosinüs benzerliğini** kullanarak hesaplayacağız.
* #### Kosinüs skoru büyüklükten bağımsızdır.
* #### Hesaplanması bir tık daha kolaydır.
* #### Formül: **cosine(x,y) = (x.y^T) / (||x|| * ||y||)**
* #### Kosinüs benzerlik matrisini hesaplıyoruz;

In [21]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.81357273, 0.15892789, ..., 0.41950984, 0.        ,
        0.53762513],
       [0.81357273, 1.        , 0.        , ..., 0.51563901, 0.        ,
        0.66081999],
       [0.15892789, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.41950984, 0.51563901, 0.        , ..., 1.        , 0.        ,
        0.55606527],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.53762513, 0.66081999, 0.        , ..., 0.55606527, 0.        ,
        1.        ]])

* #### Film başlığını giriş olarak alan ve en çok benzeyen 10 filmi veren bir işlev çıkaracağız.

In [22]:
indices = pd.Series(df.index, index = df["title"]).drop_duplicates()
indices

title
Toy Story (1995)                          0
Jumanji (1995)                            1
Grumpier Old Men (1995)                   2
Waiting to Exhale (1995)                  3
Father of the Bride Part II (1995)        4
                                      ...  
Kein Bund für's Leben (2007)          26739
Feuer, Eis & Dosenbier (2002)         26740
The Pirates (2014)                    26741
Rentun Ruusu (2001)                   26742
Innocence (2014)                      26743
Length: 26744, dtype: int64

### Öneri fonksiyonunu tanımlayacağız. İzlenecek adımlar;
* #### Başlıkla eşleşen film dizini alınır.
* #### Bu filmle tüm filmlerin benzerlik puanları alınır.
* #### Filmler benzerlik puanına göre sıralanır.
* #### En çok benzeyen 10 filmin puanları alınır.
* #### Film dizinleri edinilir.
* #### En çok benzeyen 10 film listelenir.

In [23]:
def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df["title"].iloc[movie_indices]

* #### **Toy Story (1995)** filmine en çok benzeyen 10 filme bakalım;

In [24]:
get_recommendations("Toy Story (1995)")

2209                                           Antz (1998)
3027                                    Toy Story 2 (1999)
3663        Adventures of Rocky and Bullwinkle, The (2000)
3922                      Emperor's New Groove, The (2000)
4790                                 Monsters, Inc. (2001)
10106    DuckTales: The Movie - Treasure of the Lost La...
10978                                     Wild, The (2006)
11861                               Shrek the Third (2007)
13325                       Tale of Despereaux, The (2008)
18230    Asterix and the Vikings (Astérix et les Viking...
Name: title, dtype: object

* #### **Innocence (2014)** filmine en çok benzeyen 10 filme bakalım;

In [25]:
get_recommendations("Innocence (2014)")

26743                                     Innocence (2014)
10535         Great Yokai War, The (Yôkai daisensô) (2005)
13887                   Night of the Living Dead 3D (2006)
2281                                      King Kong (1933)
7261                                        Hellboy (2004)
7328                                    Van Helsing (2004)
9921     Zu: Warriors from the Magic Mountain (Xin shu ...
15181                                  Solomon Kane (2009)
20862                             Web of Death, The (1976)
22991                   BloodRayne: The Third Reich (2011)
Name: title, dtype: object